In [ ]:
!pip install lightgbm
!pip install xgboost
!pip install catboost
!pip install tqdm
!pip install sktime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 15.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/weather_contest/electric

/content/gdrive/MyDrive/weather_contest/electric


In [ ]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# from utils.util import *
from util import clean_column_names
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
# %reload_ext autotime

In [ ]:
org=pd.read_csv("electric_train.csv")
test = pd.read_csv("electric_test.csv")

train = org.copy()

In [ ]:
def add_date_features_train(masked_df):
  masked_df['electric_train.year'] = masked_df['electric_train.tm'].dt.year
  masked_df['electric_train.month'] = masked_df['electric_train.tm'].dt.month
  masked_df['electric_train.day'] = masked_df['electric_train.tm'].dt.day
  masked_df['electric_train.quarter'] = masked_df['electric_train.tm'].dt.quarter
  masked_df['week'] = masked_df['electric_train.tm'].dt.isocalendar().week.astype(np.int32)
  return masked_df

def add_date_features_test(masked_df):
  masked_df['year'] = masked_df['TM'].dt.year
  masked_df['month'] = masked_df['TM'].dt.month
  masked_df['day'] = masked_df['TM'].dt.day
  masked_df['quarter'] = masked_df['TM'].dt.quarter
  masked_df['week'] = masked_df['TM'].dt.isocalendar().week.astype(np.int32)
  return masked_df

test['TM'] = pd.to_datetime(test['TM'])

train['electric_train.tm'] = pd.to_datetime(train['electric_train.tm'])

In [ ]:
train= add_date_features_train(train)
test= add_date_features_test(test)


In [ ]:
from scipy import stats

outlier_base = 7

# 전력기상지수의 이상치 탐지 (예제로 Z-score 방법 사용)
z_scores = np.abs(stats.zscore(train['electric_train.elec']))
outliers = train[z_scores > outlier_base]

# 지점번호별 월 평균 전력기상지수 계산
mean_by_station_and_month = train.groupby(['electric_train.stn', 'electric_train.month'])['electric_train.elec'].mean().reset_index()
mean_by_station_and_month.rename(columns={'electric_train.elec': 'electric_train.elec_month'}, inplace=True)

# 원본 데이터와 월별 평균값을 결합 (merge)
train = train.reset_index()
train = pd.merge(train, mean_by_station_and_month, on=['electric_train.stn', 'electric_train.month'], how='left')

# 이상치를 월별 평균값으로 대체
train['electric_train.elec'] = np.where(z_scores > outlier_base, train['electric_train.elec_month'], train['electric_train.elec'])

# 필요에 따라 사용된 임시 열 제거
train.drop(columns=['electric_train.elec_month','index'], inplace=True)

In [ ]:
train

,electric_train.num,electric_train.tm,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,...,electric_train.nph_rn_60m,electric_train.nph_ta_chi,electric_train.weekday,electric_train.week_name,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,week
0,4821,2021-01-01 01:00:00,1,11,884,6950,751.32,68.606449,2.2,62.7,...,0.0,-1.0,4,0,99.56,2021,1,1,1,53
1,4821,2021-01-01 02:00:00,2,11,884,6950,692.60,68.606449,2.3,63.1,...,0.0,-0.6,4,0,91.78,2021,1,1,1,53
2,4821,2021-01-01 03:00:00,3,11,884,6950,597.48,68.606449,2.2,62.4,...,0.0,-1.3,4,0,79.17,2021,1,1,1,53
3,4821,2021-01-01 04:00:00,4,11,884,6950,553.48,68.606449,1.7,63.5,...,0.0,-0.2,4,0,73.34,2021,1,1,1,53
4,4821,2021-01-01 05:00:00,5,11,884,6950,526.24,68.606449,1.7,63.0,...,0.0,-0.8,4,0,69.73,2021,1,1,1,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7593350,20947,2022-12-31 20:00:00,20,23,671,34200,6779.84,225.461986,2.7,46.3,...,0.0,-0.4,5,1,130.74,2022,12,31,4,52
7593351,20947,2022-12-31 21:00:00,21,23,671,34200,6802.40,225.461986,2.6,46.8,...,0.0,-0.5,5,1,131.18,2022,12,31,4,52
7593352,20947,2022-12-31 22:00:00,22,23,671,34200,6706.68,225.461986,2.4,47.4,...,0.0,0.2,5,1,129.33,2022,12,31,4,52
7593353,20947,2022-12-31 23:00:00,23,23,671,34200,6355.88,225.461986,2.5,47.0,...,0.0,0.3,5,1,122.57,2022,12,31,4,52


In [ ]:
train['hour_sin_time'] = np.sin(2*np.pi*train['electric_train.hh24']/24)
train['hour_cos_time'] = np.cos(2*np.pi*train['electric_train.hh24']/24)



In [ ]:
test['hour_sin_time'] = np.sin(2*np.pi*test['HH24']/24)
test['hour_cos_time'] = np.cos(2*np.pi*test['HH24']/24)


In [ ]:

train['THI'] = 1.8*train['electric_train.nph_ta'] - 0.55*(1-(train['electric_train.nph_hm']/100))*(1.8*train['electric_train.nph_ta']-26) + 32
test['THI'] = 1.8*test['nph_ta'] - 0.55*(1-(test['nph_hm']/100))*(1.8*test['nph_ta']-26) + 32

In [ ]:
def CDH(xs):
    ys = []
    for i in range(len(xs)):
            ys.append(np.sum(xs[i]-18))
    return np.array(ys)

In [ ]:
cdhs = np.array([])
cdh = CDH(train['electric_train.nph_ta'].values)
cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

In [ ]:
cdhs_ = np.array([])
cdh_ = CDH(test['nph_ta'].values)
cdhs = np.concatenate([cdhs_, cdh_])
test['CDH'] = cdhs

In [ ]:
# df = clean_column_names(train)

df=train.copy()
df

df1=test.copy()
df1

,NUM,TM,HH24,STN,nph_ta,nph_hm,nph_ws_10m,nph_rn_60m,nph_ta_chi,weekday,...,elect,year,month,day,quarter,week,hour_sin_time,hour_cos_time,THI,CDH
0,4816,2023-01-01 01:00:00,1,752,3.0,68.6,2.9,0.0,-0.1,6,...,NaN,2023,1,1,1,52,2.588190e-01,0.965926,40.957620,-15.0
1,4816,2023-01-01 02:00:00,2,752,3.1,69.4,2.7,0.0,0.3,6,...,NaN,2023,1,1,1,52,5.000000e-01,0.866025,41.016686,-14.9
2,4816,2023-01-01 03:00:00,3,752,3.6,68.3,2.3,0.0,1.2,6,...,NaN,2023,1,1,1,52,7.071068e-01,0.707107,41.883312,-14.4
3,4816,2023-01-01 04:00:00,4,752,4.0,69.2,3.1,0.0,1.1,6,...,NaN,2023,1,1,1,52,8.660254e-01,0.500000,42.384720,-14.0
4,4816,2023-01-01 05:00:00,5,752,4.2,69.5,2.5,0.0,2.0,6,...,NaN,2023,1,1,1,52,9.659258e-01,0.258819,42.653310,-13.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829473,12322,2023-12-31 20:00:00,20,901,5.6,70.8,3.2,0.0,5.8,6,...,NaN,2023,12,31,4,52,-8.660254e-01,0.500000,44.636752,-12.4
2829474,12322,2023-12-31 21:00:00,21,901,5.3,69.1,3.6,0.0,4.5,6,...,NaN,2023,12,31,4,52,-7.071068e-01,0.707107,44.337377,-12.7
2829475,12322,2023-12-31 22:00:00,22,901,5.1,70.6,3.0,0.0,5.1,6,...,NaN,2023,12,31,4,52,-5.000000e-01,0.866025,43.899794,-12.9
2829476,12322,2023-12-31 23:00:00,23,901,5.2,69.7,3.8,0.0,5.1,6,...,NaN,2023,12,31,4,52,-2.588190e-01,0.965926,44.133056,-12.8


# train set과 valid set

- train set: 2020년, 2021년
- valid set: 2022년

In [ ]:
df.set_index('electric_train.tm').sort_index()

# df1.set_index('TM').sort_index()

,electric_train.num,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,...,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,week,hour_sin_time,hour_cos_time,THI,CDH
electric_train.tm,,,,,,,,,,,,,,,,,,,,,
2020-01-01 01:00:00,13615,1,39,140,63850,9732.96,250.490543,-8.5,74.5,0.9,...,99.63,2020,1,1,1,1,2.588190e-01,0.965926,22.492325,-26.5
2020-01-01 01:00:00,18235,1,20,565,14750,2401.12,116.010234,-8.8,25.6,1.9,...,103.49,2020,1,1,1,1,2.588190e-01,0.965926,33.280928,-26.8
2020-01-01 01:00:00,18234,1,32,565,71630,12139.16,363.252769,-8.8,25.6,1.9,...,104.43,2020,1,1,1,1,2.588190e-01,0.965926,33.280928,-26.8
2020-01-01 01:00:00,18233,1,28,512,45840,6723.84,225.924357,-6.3,31.3,2.5,...,106.29,2020,1,1,1,1,2.588190e-01,0.965926,34.768919,-24.3
2020-01-01 01:00:00,11272,1,44,941,76450,12949.24,287.250770,2.4,68.0,1.2,...,102.45,2020,1,1,1,1,2.588190e-01,0.965926,40.135680,-15.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01 00:00:00,12468,24,40,152,50075,8018.88,187.277655,7.1,60.2,4.5,...,107.05,2023,1,1,1,52,-2.449294e-16,1.000000,47.673858,-10.9
2023-01-01 00:00:00,16453,24,57,617,85000,13949.76,228.064905,-5.5,81.1,0.2,...,107.31,2023,1,1,1,52,-2.449294e-16,1.000000,25.831805,-23.5
2023-01-01 00:00:00,11418,24,36,253,63735,11472.28,296.369472,5.6,59.5,1.3,...,107.53,2023,1,1,1,52,-2.449294e-16,1.000000,45.626180,-12.4


In [ ]:
tmp = df.groupby(by='electric_train.stn')['electric_train.tm'].agg(func=['max', 'min'])
tmp['기간'] = tmp['min'].dt.strftime('%Y-%m-%d') + ' ~ ' + tmp['max'].dt.strftime('%Y-%m-%d')

tmp

,max,min,기간
electric_train.stn,,,
98,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
99,2021-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2021-01-01
101,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
104,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
106,2022-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2022-01-01
...,...,...,...
942,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
943,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01
950,2023-01-01,2020-01-01 01:00:00,2020-01-01 ~ 2023-01-01


In [ ]:
tmp['기간'].value_counts()

기간
2020-01-01 ~ 2023-01-01    158
2020-01-01 ~ 2022-01-01     12
2020-01-01 ~ 2021-01-01      6
2021-01-01 ~ 2022-01-01      5
2021-01-01 ~ 2023-01-01      3
2022-01-01 ~ 2023-01-01      1
Name: count, dtype: int64

In [ ]:
mask = tmp[tmp['기간'] == '2020-01-01 ~ 2023-01-01'].index

mask

Index([ 98, 101, 104, 108, 112, 114, 119, 127, 129, 131,
       ...
       937, 938, 939, 940, 941, 942, 943, 950, 974, 991],
      dtype='int64', name='electric_train.stn', length=158)

In [ ]:
data = df[df['electric_train.stn'].isin(mask)]
data = data.reset_index(drop=True)

# df1=df1.reset_index(drop=True)

In [ ]:

data

,electric_train.num,electric_train.tm,electric_train.hh24,electric_train.n,electric_train.stn,electric_train.sum_qctr,electric_train.sum_load,electric_train.n_mean_load,electric_train.nph_ta,electric_train.nph_hm,...,electric_train.elec,electric_train.year,electric_train.month,electric_train.day,electric_train.quarter,week,hour_sin_time,hour_cos_time,THI,CDH
0,5565,2020-01-01 01:00:00,1,58,184,42250,4468.72,79.776051,2.3,65.8,...,96.58,2020,1,1,1,1,2.588190e-01,0.965926,40.251866,-15.7
1,5565,2020-01-01 02:00:00,2,58,184,42250,4101.00,79.776051,2.5,69.7,...,88.63,2020,1,1,1,1,5.000000e-01,0.866025,40.082975,-15.5
2,5565,2020-01-01 03:00:00,3,58,184,42250,3724.60,79.776051,2.8,69.5,...,80.50,2020,1,1,1,1,7.071068e-01,0.707107,40.556040,-15.2
3,5565,2020-01-01 04:00:00,4,58,184,42250,3484.40,79.776051,2.8,66.0,...,75.31,2020,1,1,1,1,8.660254e-01,0.500000,40.959520,-15.2
4,5565,2020-01-01 05:00:00,5,58,184,42250,3409.12,79.776051,3.0,66.1,...,73.68,2020,1,1,1,1,9.659258e-01,0.258819,41.240870,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7198670,20947,2022-12-31 20:00:00,20,23,671,34200,6779.84,225.461986,2.7,46.3,...,130.74,2022,12,31,4,52,-8.660254e-01,0.500000,43.103699,-15.3
7198671,20947,2022-12-31 21:00:00,21,23,671,34200,6802.40,225.461986,2.6,46.8,...,131.18,2022,12,31,4,52,-7.071068e-01,0.707107,42.918232,-15.4
7198672,20947,2022-12-31 22:00:00,22,23,671,34200,6706.68,225.461986,2.4,47.4,...,129.33,2022,12,31,4,52,-5.000000e-01,0.866025,42.592024,-15.6
7198673,20947,2022-12-31 23:00:00,23,23,671,34200,6355.88,225.461986,2.5,47.0,...,122.57,2022,12,31,4,52,-2.588190e-01,0.965926,42.767250,-15.5


In [ ]:
def clean_column_name_train(df: pd.DataFrame, prefix_to_remove: str='electric_train.') -> pd.DataFrame:
    """
    데이터프레임의 칼럼명에서 접두사를 제거하고, 칼럼명을 매핑에 따라 변경.

    Args:
        df (pd.DataFrame): 원본 데이터프레임

    Returns:
        pd.DataFrame: 칼럼명이 정리된 데이터프레임
    """

    mapping = {
    'tm': 'TM', 'hh24': 'HH24', 'weekday': 'weekday', 'week_name': 'week_name', 'sum_qctr': 'sum_qctr',
    'n': 'n', 'sum_load': 'sum_load', 'n_mean_load': 'n_mean_load', 'elec': 'elec',
    'num': 'NUM', 'stn': 'STN', 'nph_ta': 'nph_ta', 'nph_hm': 'nph_hm', 'nph_ws_10m': 'nph_ws_10m',
    'nph_rn_60m': 'nph_rn_60m', 'nph_ta_chi': 'nph_ta_chi'
    }

    # 접두사 제거
    df.columns = [col.replace(prefix_to_remove, '') for col in df.columns]

    # 칼럼명 변경
    df = df.rename(columns=mapping)

    return df

In [ ]:
data=clean_column_name_train(data)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7198675 entries, 0 to 7198674
Data columns (total 25 columns):
 #   Column         Dtype         
---  ------         -----         
 0   NUM            int64         
 1   TM             datetime64[ns]
 2   HH24           int64         
 3   n              int64         
 4   STN            int64         
 5   sum_qctr       int64         
 6   sum_load       float64       
 7   n_mean_load    float64       
 8   nph_ta         float64       
 9   nph_hm         float64       
 10  nph_ws_10m     float64       
 11  nph_rn_60m     float64       
 12  nph_ta_chi     float64       
 13  weekday        int64         
 14  week_name      int64         
 15  elec           float64       
 16  year           int32         
 17  month          int32         
 18  day            int32         
 19  quarter        int32         
 20  week           int32         
 21  hour_sin_time  float64       
 22  hour_cos_time  float64       
 23  THI    

In [ ]:
train_data, valid_data = data[data['TM'].dt.year < 2022], data[data['TM'].dt.year >= 2022]

features=train_data.columns.tolist()
# features.remove('month')
# features.remove('day')
# features.remove('quarter')
# features.remove('HH24')
features.remove('elec')
features.remove('TM')
features.remove('n')
features.remove('sum_qctr')
features.remove('sum_load')
features.remove('n_mean_load')

feature=test.columns.tolist()

feature.remove('elect')
feature.remove('TM')
# features.remove('month')
# features.remove('day')
# features.remove('quarter')
# features.remove('HH24')


In [ ]:
feature

['NUM',
 'HH24',
 'STN',
 'nph_ta',
 'nph_hm',
 'nph_ws_10m',
 'nph_rn_60m',
 'nph_ta_chi',
 'weekday',
 'week_name',
 'year',
 'month',
 'day',
 'quarter',
 'week',
 'hour_sin_time',
 'hour_cos_time',
 'THI',
 'CDH']

In [ ]:
features

['NUM',
 'HH24',
 'STN',
 'nph_ta',
 'nph_hm',
 'nph_ws_10m',
 'nph_rn_60m',
 'nph_ta_chi',
 'weekday',
 'week_name',
 'year',
 'month',
 'day',
 'quarter',
 'week',
 'hour_sin_time',
 'hour_cos_time',
 'THI',
 'CDH']

In [ ]:

# features = ['electric_train.n', 'electric_train.sum_qctr', 'electric_train.sum_load', 'electric_train.n_mean_load', 'electric_train.nph_ta', 'electric_train.nph_hm', 'electric_train.nph_ws_10m', 'electric_train.nph_rn_60m', 'electric_train.nph_ta_chi', 'electric_train.year', 'electric_train.month', 'electric_train.day', 'electric_train.hh24', 'electric_train.weekday', 'electric_train.quarter', 'electric_train.stn']
X_train = train_data[features]
y_train = train_data['elec']
X_valid = valid_data[features]
y_valid = valid_data['elec']
X_test= test[feature]


categorical_features = ['weekday', 'quarter', 'STN']
numeric_features = [feature for feature in features if features not in categorical_features]


numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 칼럼 변환기 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('NUM', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 변환기 적용
X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
X_test = preprocessor.transform(X_test)


print(X_train.shape)
print(X_valid.shape)

(4894519, 188)
(2304156, 188)


In [ ]:
print(X_test.shape)

(2829478, 188)


In [ ]:
from util import get_rscore

In [ ]:
X_train_dense = X_train.toarray()
X_valid_dense = X_valid.toarray()
X_test_dense= X_test.toarray()

In [ ]:
cat_params = {'learning_rate': 0.11501,
      'loss_function': 'RMSE',
      'verbose': False,
      'n_estimators': 650,
      'early_stopping_rounds': 100,
      'objective': 'MAE',
      'random_seed': 42,
      }

CAT_model = CatBoostRegressor(**cat_params)
CAT_model.fit(X_train_dense,y_train)

In [ ]:
# 예측
y_pred2 = CAT_model.predict(X_valid_dense)

# 성능 평가
mse2 = mean_squared_error(y_valid, y_pred2)
r_score2 = get_rscore(y_valid, y_pred2)

print(f'Mean Squared Error: {mse2}')
print(f'R Score: {r_score2}')

Mean Squared Error: 42.97643686294145
R Score: 0.967134560319253


In [ ]:
CAT_answer = CAT_model.predict(X_test_dense)

In [ ]:
test['elect'] = CAT_answer

In [ ]:
test=test.drop(columns=['year', 'month','day','quarter','hour_sin_time','hour_cos_time', 'THI','CDH'])

In [ ]:
test.to_csv("240312_0966742_CAT.csv", index=False)